In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
filename = "/content/drive/MyDrive/CN Project/InputOutput.csv"
df = pd.read_csv(filename)
print(df.info())

In [ ]:
df.head()

In [ ]:
df.plot(x="DATE",y="NET_BOOKINGS")

In [ ]:
close_data = df['NET_BOOKINGS'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['DATE'][:split]
date_test = df['DATE'][split:]

print(len(close_train))
print(len(close_test))

In [7]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 400
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

In [12]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['DATE'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [ ]:
!pip install chart_studio

import chart_studio.plotly as py
import plotly.graph_objs as go

prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Airline Data",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Number of Bookings"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

We have divided 1 year data into two parts: Training (from Jan to Oct) and Testing(Oct to Dec). In the time from Oct to Dec, 4th Nov 2018 is the actual best day of buying tickets as it has the least number of bookings of 55 in this time frame. 
The model predicted the day to be 2nd Nov 2018, which would have minimum bookings of nearly 54, which is quite close to the actual decision one would have made. 
The model prediction curve also nearly overlaps with the ground truth data with an MSE loss of 1750.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(forecast_dates,forecast)

This is my prediction for the number of bookings in the month of January 2019